<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_PAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
# 첫 설치 이후 설치 안함
# !pip install yfinance

In [180]:
# 첫 설치 이후 설치 안함
# !pip install pandas_market_calendars

In [181]:
from pandas_datareader import data as pdr
import yfinance as yf

# yfinance 라이브러리 내장 기능을 활성화한다.
# yfinance의 데이터 가져오기 기능이 pandas_datareader 를 통해 사용할 수 있다.
yf.pdr_override()

import pandas as pd
import exchange_calendars as ecals
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 11)

# ETF 가격 지정된 기간에서 가져와서 DataFrame에 담기
'''
def get_price_data(x, start_date, end_date):
  df_price = pd.DataFrame(columns=x)
  for ticker in x:
    df_price[ticker] = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)['Adj Close']
  return df_price.dropna(axis=0)
'''

# 1년 전 개장일 구하기
us_market_code = "XNYS"
us_calendar = ecals.get_calendar(us_market_code)

# 각 날짜 변수에 저장
today = datetime.today().date()
today_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=0)).date())
month_01_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=30)).date())
month_03_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=91)).date())
month_06_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=182)).date())
month_12_ago_workday = pd.Timestamp(us_calendar.previous_open(today - timedelta(days=365)).date())

# 전략에 사용할 ETF 명 리스트에 담기
PAA = ['SPY', 'QQQ', 'IWM', 'VGK', 'EWJ', 'EEM', 'IYR', 'GSG', 'GLD', 'HYG', 'LQD', 'TLT', 'IEF']

# ETF 가격 추출
# df_PAA = get_price_data(PAA, month_12_ago_workday, today).sort_index(ascending=False)
# [PAA] : PAA 순서대로 만들기
df_PAA = yf.download(PAA, start=month_12_ago_workday, end=today)['Adj Close'].sort_index(ascending=False)[PAA]
display(df_PAA)

[*********************100%***********************]  13 of 13 completed


,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
Date,,,,,,,,,,,,,
2023-08-14,448.1100,370.3500,190.6600,61.5800,60.8200,39.3700,85.4900,21.4400,177.0600,74.8200,105.0100,95.1600,93.9800
2023-08-11,445.6500,366.2400,190.9900,61.7000,61.3700,39.5600,86.0900,21.5600,177.6000,74.8300,105.0500,95.3700,94.1300
2023-08-10,445.9100,368.5900,190.7200,62.2300,61.4700,40.1500,85.9200,21.5900,177.6000,74.9400,105.4100,95.5900,94.6300
2023-08-09,445.7500,367.9100,191.5700,61.9900,61.2100,40.0900,86.2500,21.8000,177.7900,74.9600,106.1600,97.1900,95.3600
2023-08-08,448.7500,372.0000,193.0800,61.8500,61.7500,39.9900,86.2100,21.5000,178.6400,74.9500,106.0600,96.6900,95.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-18,421.0149,326.9490,195.7213,53.2366,56.0968,39.5330,98.6046,22.3600,163.9200,73.3729,108.6893,111.5009,100.7491
2022-08-17,419.7948,326.1646,194.4323,53.4979,56.2950,39.7578,99.3714,22.0200,164.3500,73.2596,108.4102,111.3359,100.5541
2022-08-16,422.7958,329.9277,197.7580,54.0495,56.4733,39.9240,99.7887,21.8600,165.4200,74.0241,109.4399,112.5102,101.2364


In [182]:
# 12개월 이동평균 생성 및 저장
PAA_12mo_ma = df_PAA.rolling(window=252).mean().loc[month_12_ago_workday]  # 252 개의 영업일은 대략 12개월
df_PAA_12mo_ma = pd.DataFrame([PAA_12mo_ma], columns=PAA, index=['12mo_moving_average'])

# 가격 비율 계산 및 DataFrame 생성
df_PAA_12mo_ror = pd.DataFrame(index=['12개월 수익률'])
for symbol in PAA:
    price_today = df_PAA[symbol][0]
    price_12mo_ma = PAA_12mo_ma[symbol]
    ror = (price_today / price_12mo_ma - 1) * 100
    df_PAA_12mo_ror[symbol] = [ror]

df_PAA_12mo_ma = pd.concat([df_PAA_12mo_ma, df_PAA_12mo_ror])

# csv 경로 설정
output_path =  '/content/drive/MyDrive/colab/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} PAA.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

str_PAA_12mo_ma = str_today + "### 12개월 이동평균 및 12개월 수익률"
display(Markdown(str_PAA_12mo_ma))
display(df_PAA_12mo_ma)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'w') as f:
  f.write(f'{str_PAA_12mo_ma}\n')
  df_PAA_12mo_ma.to_csv(f, index=True)

# 0 이하 원소 개수 세기
count_negative_etf = (df_PAA_12mo_ror < 0).sum().sum().item()
str_count_negative_etf = "### 12개월 수익률 0 미만 ETF 개수: " + str(count_negative_etf)

display(Markdown(str_count_negative_etf))

# 안전자산 비율 계산
display(Markdown('### 안전자산 비율: ' + str(round((count_negative_etf) * (100/6), 1)) + '%'))

### 2023-08-15 ### 12개월 이동평균 및 12개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
12mo_moving_average,403.2125,309.9914,180.8306,56.5855,56.3974,38.4234,85.6225,20.7340,172.3350,72.3846,105.0953,101.8544,96.1617
12개월 수익률,11.1349,19.4710,5.4357,8.8264,7.8419,2.4636,-0.1548,3.4052,2.7418,3.3645,-0.0811,-6.5725,-2.2688


### 12개월 수익률 0 미만 ETF 개수: 4

### 안전자산 비율: 66.7%

In [183]:
# 모멘텀스코어 계산할 오늘-1-3-6-12개월 날짜 저장
ago_name = ['today', 'month_01_ago', 'month_03_ago', 'month_06_ago', 'month_12_ago']
ago_date = [today_workday.date(), \
            month_01_ago_workday.date(), \
            month_03_ago_workday.date(), \
            month_06_ago_workday.date(), \
            month_12_ago_workday.date()]

# '개월 전/날짜' 형식으로 텍스트 저장
ago = [f'{a}/{d}' for a, d in zip(ago_name, ago_date)]

# ETF의 날짜별 데이터 담기
etf_data = {}

for i in range(len(PAA)):
  etf_data[PAA[i]] = [df_PAA.loc[today_workday, PAA[i]], \
                          df_PAA.loc[month_01_ago_workday, PAA[i]], \
                          df_PAA.loc[month_03_ago_workday, PAA[i]], \
                          df_PAA.loc[month_06_ago_workday, PAA[i]], \
                          df_PAA.loc[month_12_ago_workday, PAA[i]]]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
df_etf_price = pd.DataFrame(etf_data, columns=PAA, index=ago)
str_etf_price = str_today + "### ETF 가격 현재-1-3-6-12 개월"

display(Markdown(str_etf_price))
display(df_etf_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_price}\n')
  df_etf_price.to_csv(f, index=True)

### 2023-08-15 ### ETF 가격 현재-1-3-6-12 개월

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
today/2023-08-14,448.1100,370.3500,190.6600,61.5800,60.8200,39.3700,85.4900,21.4400,177.0600,74.8200,105.0100,95.1600,93.9800
month_01_ago/2023-07-14,449.2800,379.0700,191.5300,63.1700,62.4200,41.0100,89.1500,20.3800,181.4300,74.7439,107.0824,101.0116,96.2860
month_03_ago/2023-05-15,411.4815,326.3423,174.3472,61.9255,60.1460,38.9548,83.6352,19.3200,187.2100,73.2989,106.6788,102.3564,98.5277
month_06_ago/2023-02-13,409.7384,303.6129,191.3620,59.6484,57.6928,40.2050,91.3263,20.8300,172.4800,72.7660,106.0275,102.6436,95.8646
month_12_ago/2022-08-12,420.2376,328.0511,197.1381,54.5237,56.5130,40.1194,99.6723,22.5700,167.8700,74.5055,109.9403,112.1414,101.0317


In [184]:
etf_momentum_score = []
etf_ror_data = []

## DataFrame 만들기: 수익률
for etf_name in PAA:
  etf_today_price = df_etf_price.loc[ago[0], etf_name]
  etf_momentum_score.append((12 * (etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100) + \
                            (4 * (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100) + \
                            (2 * (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100) + \
                            (1 * (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100))

  etf_ror_data.append([(etf_today_price/df_etf_price.loc[ago[1], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[2], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[3], etf_name] - 1) * 100, \
                       (etf_today_price/df_etf_price.loc[ago[4], etf_name] - 1) * 100])

df_etf_ror = pd.DataFrame(etf_ror_data, columns=ago[1:], index=PAA).transpose()

str_etf_ror = str_today + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
  # df_etf_ror.to_csv(filename, mode='a', header=True, index=True)

### 2023-08-15 ### ETF 1-3-6-12 개월 수익률

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-07-14,-0.2604,-2.3004,-0.4542,-2.5170,-2.5633,-3.9990,-4.1054,5.2012,-2.4086,0.1018,-1.9353,-5.7930,-2.3949
month_03_ago/2023-05-15,8.9016,13.4851,9.3565,-0.5580,1.1207,1.0658,2.2178,10.9731,-5.4217,2.0752,-1.5643,-7.0307,-4.6157
month_06_ago/2023-02-13,9.3649,21.9810,-0.3668,3.2383,5.4204,-2.0769,-6.3907,2.9285,2.6554,2.8227,-0.9596,-7.2909,-1.9658
month_12_ago/2022-08-12,6.6325,12.8940,-3.2861,12.9417,7.6213,-1.8680,-14.2289,-5.0066,5.4745,0.4221,-4.4846,-15.1429,-6.9797


In [185]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 12 if row.name == ago[1] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 4 if row.name == ago[2] else row, axis=1)
df_etf_ror_weight = df_etf_ror_weight.apply(lambda row: row * 2 if row.name == ago[3] else row, axis=1)

str_etf_ror_weight = str_today + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-08-15 ### ETF 1-3-6-12 개월 가중치 적용

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
month_01_ago/2023-07-14,-3.1250,-27.6044,-5.4508,-30.2041,-30.7593,-47.9883,-49.2653,62.4142,-28.9037,1.2218,-23.2236,-69.5159,-28.7394
month_03_ago/2023-05-15,35.6064,53.9405,37.4261,-2.2318,4.4827,4.2632,8.8710,43.8924,-21.6869,8.3008,-6.2572,-28.1230,-18.4627
month_06_ago/2023-02-13,18.7298,43.9620,-0.7337,6.4766,10.8407,-4.1538,-12.7813,5.8569,5.3108,5.6454,-1.9193,-14.5818,-3.9317
month_12_ago/2022-08-12,6.6325,12.8940,-3.2861,12.9417,7.6213,-1.8680,-14.2289,-5.0066,5.4745,0.4221,-4.4846,-15.1429,-6.9797


In [186]:
## DataFrame 만들기: 모멘텀
df_etf_momentum_score = pd.DataFrame(etf_momentum_score, columns=['momentum_score'], index=PAA).transpose()

str_etf_momentum_score = str_today + "### ETF 별 12개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-08-15 ### ETF 별 12개월 Momentum Score

,SPY,QQQ,IWM,VGK,EWJ,EEM,IYR,GSG,GLD,HYG,LQD,TLT,IEF
momentum_score,57.8437,83.1921,27.9555,-13.0177,-7.8146,-49.7469,-67.4045,107.1569,-39.8053,15.5902,-35.8846,-127.3635,-58.1134


In [187]:
# 12개월 수익률 (-)인 ETF 개수 및 16.7%씩 투자할 자산 개수와 종목
count_invest_etf = (lambda count_negative_etf: 6 - count_negative_etf if count_negative_etf < 7 else 0)(count_negative_etf)
str_count_invest_etf = "## 16.7% 씩 투자할 자산 개수: " + str(count_invest_etf)

copy_etf_momentum_score = etf_momentum_score.copy()
recommended_etfs = []
for i in range(count_invest_etf):
  recommended_etfs.append(PAA[copy_etf_momentum_score.index(max(copy_etf_momentum_score))])
  copy_etf_momentum_score[copy_etf_momentum_score.index(max(copy_etf_momentum_score))] = -99999  # 선택된 ETF 점수를 낮은 값으로 변경

str_etf_PAA = "## 및 추천 종목: " + ' '.join(recommended_etfs)

display(Markdown(str_today[1:]))
display(Markdown(str_count_negative_etf[1:]))
display(Markdown(str_count_invest_etf))
display(Markdown(str_etf_PAA))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:]}\n')
  f.write(f'{str_count_negative_etf[1:]}\n')
  f.write(f'{str_count_invest_etf}\n')
  f.write(f'{str_etf_PAA}')

## 2023-08-15 

## 12개월 수익률 0 미만 ETF 개수: 4

## 16.7% 씩 투자할 자산 개수: 2

## 및 추천 종목: GSG QQQ